# Facem o clasificarea a florilor cu VGG16
## Incarcam modelul VGG16


In [1]:
from keras.applications import VGG16
#VGG16 a fost proiectat sa functioneze cu imagini cu dimensiunea de 224x224 pixel
img_rows = 224
img_cols = 224

#Incarcare model VGG16
vgg16 = VGG16(weights = "imagenet",
              include_top = False,
              input_shape = (img_rows, img_cols, 3)  
)

Instructions for updating:
Colocations handled automatically by placer.


In [2]:
#printeaza layers
for (i, layer) in enumerate(vgg16.layers):
    print(str(i)+ " "+layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


# Incheata toate layers cu exceptia primelor 4 de sus


In [3]:
#VGG16 a fost proiectat sa functioneze cu imagini cu dimensiunea de 224x224 pixel
img_rows = 224
img_cols = 224

#Reîncărcați modelul VGG16 fără straturile superioare sau FC
vgg16 =VGG16(weights = 'imagenet',
             include_top = False,
             input_shape = (img_rows, img_cols, 3)
)

#Aici vom incheta ultimele 4 straturi/layers
#În mod implicit, straturile sunt instruibile ca True
for layer in vgg16.layers:
    layer.trainable = False

#printeaza layers
for (i, layer) in enumerate(vgg16.layers):
    print(str(i)+ " "+layer.__class__.__name__, layer.trainable)


0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


# Cream o functie care returneaza capul FC


In [4]:
def addTopModel(botton_model, num_classes, D=256):
    """creează partea superioară sau capul modelului care va fi plasat pe partea de sus a straturilor de jos"""
    top_model = botton_model.output
    top_model = Flatten(name= "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation= "softmax")(top_model)
    return top_model

# Adauga capul FC inapoi in VGG

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 17
FC_Head = addTopModel(vgg16, num_classes)
model =Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

# incarca baza de date cu florii

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'c:/Users/Harum/Documents/15/17_flowers/train'
validation_data_dir = 'c:/Users/Harum/Documents/15/17_flowers/validation'

train_datagen = ImageDataGenerator(
                   rescale=1./255,
                   rotation_range=20,
                   width_shift_range=0.2,
                   height_shift_range=0.2,
                   horizontal_flip=True,
                   fill_mode='nearest'
)
validation_datagen = ImageDataGenerator(rescale=1./255)

#Schimba batchsize in concordanta cu memoria RAM in sistem
train_batchsize = 16
val_batchsize = 10

train_generator = train_datagen.flow_from_directory(
                   train_data_dir,
                   target_size=(img_rows, img_cols),
                   batch_size=train_batchsize,
                   class_mode='categorical'
)
validations_generator = validation_datagen.flow_from_directory(
                         validation_data_dir,
                         target_size=(img_rows, img_cols),
                         batch_size=val_batchsize,
                         class_mode= 'categorical',
                         shuffle=False
)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


# Instruire stratul/layers de sus

In [13]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("c:/Users/Harum/Documents/15/flowers_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1
)

earlystop = EarlyStopping(monitor= 'val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights=True
                         )

#pune  call backs intrun callback list
callbacks = [earlystop, checkpoint]

#Utilizam o rata de invatare doarte mica
model.compile(RMSprop(0.001),'categorical_crossentropy', ['accuracy'])

#Introduce numarul de instruire si validare 
nb_train_samples = 1190
nb_validation_samples = 170
#parametri de instruire
epochs = 10
batch_size =64

# instruire model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs= epochs,
    callbacks=callbacks,
    validation_data=validations_generator,
    validation_steps=nb_validation_samples // batch_size

)



Epoch 1/10
18/18 [==============================] - 5s 263ms/step - loss: 1.2212 - accuracy: 0.6978 - val_loss: 0.6690 - val_accuracy: 0.7500

Epoch 00001: val_loss improved from inf to 0.66903, saving model to c:/Users/Harum/Documents/15/flowers_vgg.h5
Epoch 2/10
18/18 [==============================] - 4s 227ms/step - loss: 0.9020 - accuracy: 0.6910 - val_loss: 1.4032 - val_accuracy: 0.7500

Epoch 00002: val_loss did not improve from 0.66903
Epoch 3/10
18/18 [==============================] - 4s 242ms/step - loss: 0.8170 - accuracy: 0.7812 - val_loss: 0.2575 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.66903 to 0.25750, saving model to c:/Users/Harum/Documents/15/flowers_vgg.h5
Epoch 4/10
18/18 [==============================] - 4s 239ms/step - loss: 0.8256 - accuracy: 0.7292 - val_loss: 0.0566 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.25750 to 0.05663, saving model to c:/Users/Harum/Documents/15/flowers_vgg.h5
Epoch 5/10
18/18 [==================

# Putem mari viteza modelului
 sa incercam sa scadem dimensiunea imagini la 64x64

In [14]:
#Setam dimensiunea inputului la 64 x 64 pixeli
img_rows = 64
img_cols = 64

#Reîncărcați modelul VGG16 fără straturile superioare sau FC
vgg16 = VGG16(weights = 'imagenet',
            include_top = False,
            input_shape = (img_rows, img_cols, 3)
 )

#Aici vom incheta ultimele 4 straturi/layers
#În mod implicit, straturile sunt instruibile ca True
for layer in vgg16.layers:
    layer.trainable = False

#printeaza layers
for (i, layer) in enumerate(vgg16.layers):
    print(str(i)+ " "+layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


# Cream un noe model folosind o imagine a dimensiuni de 64 x 64

In [17]:
from keras.applications import VGG16
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'c:/Users/Harum/Documents/15/17_flowers/train'
validation_data_dir = 'c:/Users/Harum/Documents/15/17_flowers/validation'

train_datagen = ImageDataGenerator(
                   rescale=1./255,
                   rotation_range=20,
                   width_shift_range=0.2,
                   height_shift_range=0.2,
                   horizontal_flip=True,
                   fill_mode='nearest'
)
validation_datagen = ImageDataGenerator(rescale=1./255)

#Schimba batchsize in concordanta cu memoria RAM in sistem
train_batchsize = 16
val_batchsize = 10

train_generator = train_datagen.flow_from_directory(
                   train_data_dir,
                   target_size=(img_rows, img_cols),
                   batch_size=train_batchsize,
                   class_mode='categorical'
)
validations_generator = validation_datagen.flow_from_directory(
                         validation_data_dir,
                         target_size=(img_rows, img_cols),
                         batch_size=val_batchsize,
                         class_mode= 'categorical',
                         shuffle=False)

#Reîncărcați modelul VGG16 fără straturile superioare sau FC
vgg16 = VGG16(weights = 'imagenet',
            include_top = False,
            input_shape = (img_rows, img_cols, 3)
 )

#Aici vom incheta ultimele 4 straturi/layers
#În mod implicit, straturile sunt instruibile ca True
for layer in vgg16.layers:
    layer.trainable = False

#numarul de clase in baza de date Flowers-17
num_classes =17

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
________________________________________

# instruire folosind imagini cu dimensiunea de 64 x 64 este mult mai rapid

In [21]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("c:/Users/Harum/Documents/15/flowers_vgg_64.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1
)

earlystop = EarlyStopping(monitor='val_loss',
                        min_delta=0,
                        patience=5,
                        verbose=1,
                        restore_best_weights=True
                        )

reduce_lr = ReduceLROnPlateau(monitor= 'val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              restore_best_weights = True 
                                  )

#punem call backs intro lista callbacks
callbacks = [earlystop, checkpoint, reduce_lr]

#Folosim o rata de invatare foarte mica
model.compile(RMSprop(0.0001), 'categorical_crossentropy', ['accuracy'])

#Introduce numarul de instruire si validare 
nb_train_samples = 1190
nb_validation_samples = 170

#parametri de instruire
epochs=25
batch_size = 16

# instruire
history= model.fit_generator(
    train_generator,
    steps_per_epoch= nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validations_generator,
    validation_steps= nb_validation_samples //batch_size
)


Epoch 1/25
74/74 [==============================] - 5s 70ms/step - loss: 1.9703 - accuracy: 0.4412 - val_loss: 1.7163 - val_accuracy: 0.5600

Epoch 00001: val_loss improved from inf to 1.71630, saving model to c:/Users/Harum/Documents/15/flowers_vgg_64.h5
Epoch 2/25
74/74 [==============================] - 5s 71ms/step - loss: 1.8240 - accuracy: 0.4676 - val_loss: 1.2416 - val_accuracy: 0.5200

Epoch 00002: val_loss improved from 1.71630 to 1.24156, saving model to c:/Users/Harum/Documents/15/flowers_vgg_64.h5
Epoch 3/25
74/74 [==============================] - 5s 71ms/step - loss: 1.7119 - accuracy: 0.5119 - val_loss: 1.3039 - val_accuracy: 0.6200

Epoch 00003: val_loss did not improve from 1.24156
Epoch 4/25
74/74 [==============================] - 5s 71ms/step - loss: 1.5778 - accuracy: 0.5358 - val_loss: 1.5709 - val_accuracy: 0.5200

Epoch 00004: val_loss did not improve from 1.24156
Epoch 5/25
74/74 [==============================] - 5s 71ms/step - loss: 1.5599 - accuracy: 0.5298